# NoSQL (MongoDB) (sesión 1)

## Contenidos

* Introducción
* Instalación inicial de MongoDB
* Descarga de datos CSV
* Instalación de la librería `pymongo`
* Conexión a la base de datos
* El API de colección de MongoDB
* Búsqueda de documentos
* Borrado de documentos
* Inserción de elementos
* Operadores de colección
* Ejemplos básicos de agregación
* Ejercicios




## Introducción

![MongoDB](https://webassets.mongodb.com/_com_assets/cms/MongoDB_Logo_FullColorBlack_RGB-4td3yuxzjs.png)

Esta hoja muestra cómo acceder a bases de datos MongoDB y también a conectar la salida con Jupyter. Se puede utilizar el *shell* propio de MongoDB. La diferencia es que ese programa espera código Javascript y aquí trabajaremos con Python. A continuación, se introducen los conceptos básicos para la realización de consultas sencillas en Python.


En primer lugar n unas comprobaciones iniciales.

In [31]:
RunningInCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False

In [32]:
!sudo apt-get update -qq
!sudo apt-get install -y -qq gpg p7zip

## Instalación inicial de MongoDB (no necesaria si se utiliza Docker en local)


In [33]:
!wget -qO - https://www.mongodb.org/static/pgp/server-6.0.asc | sudo bash -c 'gpg --dearmor > /etc/apt/trusted.gpg.d/mongo-server-6.gpg'

In [34]:
%%bash
sudo adduser --system --no-create-home mongodb
sudo addgroup --system mongodb
sudo adduser mongodb mongodb

# create db -- note: this should agree with dbpath in mongod.conf
if [ ! -d /var/lib/mongodb ]; then
  sudo mkdir -p /var/lib/mongodb
  sudo chown mongodb:mongodb /var/lib/mongodb
fi

# create logdir -- note: this should agree with logpath in mongod.conf
if [ ! -d /var/log/mongodb ]; then
  sudo mkdir -p /var/log/mongodb
  sudo chown mongodb:mongodb /var/log/mongodb
fi

The system user `mongodb' already exists. Exiting.
The user `mongodb' is already a member of `mongodb'.


addgroup: The group `mongodb' already exists as a system group. Exiting.


In [35]:
!echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-6.0.list

deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/6.0 multiverse


In [36]:
!sudo ln -sf /bin/true /bin/systemctl

In [37]:
!sudo apt-get update -qq
!sudo apt-get install -y -qq dialog mongodb-org

In [38]:
!sudo /usr/bin/mongod --config /etc/mongod.conf --fork

about to fork child process, waiting until server is ready for connections.
forked process: 6208
ERROR: child process failed, exited with 48
To see additional information in this output, start without the "--fork" option.


In [39]:
!mongod --version

db version v6.0.13
Build Info: {
    "version": "6.0.13",
    "gitVersion": "3b13907f9bdf6bd3264d67140d6c215d51bbd20c",
    "openSSLVersion": "OpenSSL 3.0.2 15 Mar 2022",
    "modules": [],
    "allocator": "tcmalloc",
    "environment": {
        "distmod": "ubuntu2204",
        "distarch": "x86_64",
        "target_arch": "x86_64"
    }
}


## Descarga de los datos en formato CSV

 - Formato: 7zipped
 - Ficheros:
   - **Comments**.csv
       - Id
       - PostId
       - Score
       - Text, e.g.: "@Stu Thompson: Seems possible to me - why not try it?"
       - CreationDate, e.g.:"2008-09-06T08:07:10.730"
       - UserId
   - **Posts**.csv
       - Id
       - PostTypeId
          - 1: Question
          - 2: Answer
       - ParentID (only present if PostTypeId is 2)
       - AcceptedAnswerId (only present if PostTypeId is 1)
       - CreationDate
       - Score
       - ViewCount
       - Body
       - OwnerUserId
       - LastEditorUserId
       - LastEditorDisplayName="Jeff Atwood"
       - LastEditDate="2009-03-05T22:28:34.823"
       - LastActivityDate="2009-03-11T12:51:01.480"
       - CommunityOwnedDate="2009-03-11T12:51:01.480"
       - ClosedDate="2009-03-11T12:51:01.480"
       - Title=
       - Tags=
       - AnswerCount
       - CommentCount
       - FavoriteCount
   - **Tags**.csv
    - Id
    - Count
    - ExcerptPostId
    - TagName
    - WikiPostId
   - **Users**.csv
     - Id
     - Reputation
     - CreationDate
     - DisplayName
     - EmailHash
     - LastAccessDate
     - WebsiteUrl
     - Location
     - Age
     - AboutMe
     - Views
     - UpVotes
     - DownVotes
   - **Votes**.csv
     - Id
     - PostId
     - VoteTypeId
        - ` 1`: AcceptedByOriginator
        - ` 2`: UpMod
        - ` 3`: DownMod
        - ` 4`: Offensive
        - ` 5`: Favorite - if VoteTypeId = 5 UserId will be populated
        - ` 6`: Close
        - ` 7`: Reopen
        - ` 8`: BountyStart
        - ` 9`: BountyClose
        - `10`: Deletion
        - `11`: Undeletion
        - `12`: Spam
        - `13`: InformModerator
     - CreationDate
     - UserId (only for VoteTypeId 5)
     - BountyAmount (only for VoteTypeId 9)

In [40]:
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.csv.7z.001
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.csv.7z.002

--2024-02-21 15:15:40--  https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.csv.7z.001
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dsevilla/bd2-data/main/es.stackoverflow/es.stackoverflow.csv.7z.001 [following]
--2024-02-21 15:15:41--  https://raw.githubusercontent.com/dsevilla/bd2-data/main/es.stackoverflow/es.stackoverflow.csv.7z.001
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104857600 (100M) [application/octet-stream]
Saving to: ‘es.stackoverflow.csv.7z.001.1’

es.stackoverflow.cs 100%[===================>] 100.00M  63.8MB/s    in 1.6s    

2024-02-21 1

In [41]:
!7zr x es.stackoverflow.csv.7z.001


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 104857600 bytes (100 MiB)

Extracting archive: es.stackoverflow.csv.7z.001
  0% 1 Open           --
Path = es.stackoverflow.csv.7z.001
Type = Split
Physical Size = 104857600
Volumes = 2
Total Physical Size = 200457538
----
Path = es.stackoverflow.csv.7z
Size = 200457538
--
Path = es.stackoverflow.csv.7z
Type = 7z
Physical Size = 200457538
Headers Size = 248
Method = LZMA2:24
Solid = +
Blocks = 1

  0%    
Would you like to replace the existing file:
  Path:     ./Comments.csv
  Size:     160819553 bytes (154 MiB)
  Modified: 2024-01-21 23:51:00
with the file from archive:
  Path:     Comments.csv
  Size:     160819553 bytes (154 MiB)
  Modified: 2024-01-21 23:51:00
? (Y)es / (N)o / 

In [42]:
!head Users.csv

Id,AboutMe,AccountId,CreationDate,DisplayName,DownVotes,LastAccessDate,Location,Reputation,UpVotes,Views,WebsiteUrl
-1,"<p>Hola, no soy una persona real.</p><br/><br/><p>¡Soy un proceso que ayuda a mantener el sitio limpio!</p><br/><br/><p>Hago cosas como:</p><br/><br/><ul><br/><li>Dar empujoncitos a preguntas antiguas sin respuesta aproximadamente cada hora, para que atraigan algo de atención.</li><br/><li>Tener la propiedad de las preguntas y respuestas wiki para que nadie se lleve reputación por ellas</li><br/><li>Recibir la propiedad de los votos negativos en las publicaciones de spam o dañinas que son borradas permanentemente</li><br/><li>Tener la propiedad de las ediciones sugeridas por usuarios anónimos</li><br/><li><a href=""http://meta.stackoverflow.com/a/92006"">Quitar preguntas abandonadas</a></li><br/></ul><br/>",-1,2015-10-26T21:36:24.767,Comunidad,22504,2015-10-26T21:36:24.767,en la granja de servidores,1,10211,2516,
1,"<p>Dev #2 who helped create Stack Overflow currently

## Instalación de la librería `pymongo`


In [43]:
!pip install --upgrade pymongo

In [44]:
from pprint import pprint as pp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

Usaremos la librería `pymongo` para python. La cargamos a continuación.

In [45]:
import pymongo
from pymongo import MongoClient

## Conexión a la base de datos

La conexión se inicia con `MongoClient` en el `host` descrito en el fichero `docker-compose.yml` (`mongo`), o bien a `localhost` si lo estamos haciendo en Colab.

In [46]:
db_hostname = "localhost" if RunningInCOLAB else "mongo"

In [47]:
client = MongoClient(db_hostname,27017)
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [48]:
client.list_database_names()

['admin', 'config', 'local', 'stackoverflow']

In [49]:
import csv
from datetime import datetime
from tqdm.notebook import tqdm

def batched(iterable, n):
    from itertools import islice
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while batch := tuple(islice(it, n)):
        yield batch

def csv_to_mongo(file, coll):
    """
    Carga un fichero CSV en Mongo. file especifica el fichero, coll la colección
    dentro de la base de datos, y date_cols las columnas que serán interpretadas
    como fechas.
    """
    # Convertir todos los elementos que se puedan a números
    def to_numeric(d):
        try:
            return int(d)
        except ValueError:
            try:
                return float(d)
            except ValueError:
                return d

    def to_date(d):
        """To ISO Date. If this cannot be converted, return NULL (None)"""
        try:
            return datetime.strptime(d, "%Y-%m-%dT%H:%M:%S.%f")
        except ValueError:
            return None

    def to_str(d):
        try:
          return str(d)
        except ValueError:
            return None

    coll.drop()

    with open(file, encoding='utf-8') as f:
        # La llamada csv.reader() crea un iterador sobre un fichero CSV
        reader = csv.reader(f, dialect='excel')

        # Se leen las columnas. Sus nombres se usarán para crear las diferentes columnas en la familia
        columns = next(reader)

        # Las columnas que contienen 'Date' se interpretan como fechas
        func_to_cols = list(map(lambda c: to_date if 'date' in c.lower() else to_numeric, columns))
        #func_to_cols = list(map(lambda c: to_date if 'date' in c.lower() else (to_numeric if not 'displayname' in c.lower() else to_str), columns))

        for batch in batched(tqdm(reader, desc='Leyendo e insertando filas...'), 10000):
            docs = []
            for row in batch:
                row = [func(e) for (func,e) in zip(func_to_cols, row)]
                docs.append(dict(zip(columns,row)))
            coll.insert_many(docs)

        print("¡Hecho!")

Las bases de datos se crean conforme se nombran. Se puede utilizar la notación punto o la de diccionario. Las colecciones también.

In [50]:
db = client.stackoverflow
db = client['stackoverflow']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'stackoverflow')

Ahora vemos que se ha creado una nueva BD `stackoverflow` en Mongo.

In [51]:
client.list_database_names()

['admin', 'config', 'local', 'stackoverflow']

## Colecciones en MongoDB

Las bases de datos están compuestas por un conjunto de **colecciones**. Cada colección aglutina a un conjunto de objetos (documentos) del mismo tipo, aunque como vimos en teoría, cada documento puede tener un conjunto de atributos diferente.

In [52]:
posts = db.posts
posts

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'stackoverflow'), 'posts')

Las colecciones en Mongo pueden cambiarse de nombre con la función `.rename`.

Vamos a crear una colección ficticia llamada *colleccion* para posteriormente renombrarla como *startwars*.

Al establecer el parámetro `dropTarget`a `True` forzamos a que, si ya existía una colección `starwars` sus datos se  sobreescriben por la nueva colección renombrada.

In [53]:
# creamos la colección
collection = db['coleccion']
docs = [{"id":1, "nombre":"Luke"},
        {"id":3, "nombre":"Leia"}]
collection.insert_many(docs)

# Ahora la renombramos
collection.rename('starwars', dropTarget = True)

result = db.list_collection_names()
for collect in result:
    print(collect)

starwars
comments
votes
posts
users
tags


## Importación de los ficheros CSV

Por ahora creamos una colección diferente para cada uno. Después estudiaremos cómo poder optimizar el acceso usando agregación.

In [54]:
csv_to_mongo('Posts.csv',db.posts)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

¡Hecho!


In [55]:
db.drop_collection( 'Users')
# csv_to_mongo('Users.csv',db.users)

{'ok': 0.0,
 'errmsg': 'ns not found',
 'code': 26,
 'codeName': 'NamespaceNotFound'}

¿Qué ha pasado? Un valor erróneo en atributo `DisplayName` de uno de los usuarios provoca que dicho valor se convierta a `int`. Posteriormente dicho valor no puede se almacenado por Mongo pues excede su rango de valores para int lanzando la excepción `OverflowError: MongoDB can only handle up to 8-byte ints`.

Para solucionarlo deberemos descomentar la línea `#func_to_cols = list(map(lambda c: to_date if 'date' in c.lower() else (to_numeric if not 'displayname' in c.lower() else to_str), columns))` dentro de la función `csv_to_mongo` (sin olvidar comentar la anterior).

Con ello forzamos a que el campo `DisplayName` sea siempre de tipo `str`.


In [ ]:
csv_to_mongo('Votes.csv',db.votes)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

In [28]:
csv_to_mongo('Comments.csv',db.comments)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

¡Hecho!


In [29]:
csv_to_mongo('Tags.csv',db.tags)

Leyendo e insertando filas...: 0it [00:00, ?it/s]

¡Hecho!


In [30]:
posts.count_documents({})

410346

## El API de colección de MongoDB

El API de colección en Python se puede encontrar aquí: https://docs.mongodb.com/drivers/pymongo/. La mayoría de libros y referencias muestran el uso de mongo desde Javascript, ya que el *shell* de MongoDB acepta ese lenguaje. La sintaxis con respecto a Python cambia un poco, y se puede seguir en el enlace anterior. Existe incluso un curso de la MongoDB University que se puede realizar de forma gratuita: https://learn.mongodb.com/learning-paths/using-mongodb-with-python

Con `.find_one` obtenemos el primer elemento de una colección.

In [ ]:
post = posts.find_one()
post

Utilizo la librería `pp` (*PrettyPrint*) para imprimir los objetos grandes de una manera amigable.

In [ ]:
users = db.users
pp(users.find_one())

A cada objeto se le asigna una clave implícita con nombre "`_id`" (si el objeto no lo incluye).

In [ ]:
print (type(post['_id']))
post['_id']

La siguiente sintaxis está descatalogada en las nuevas versiones, pero era más conveniente:

In [ ]:
#posts.save(post)

Ahora hay que hacerlo así (el resultado será 0 porque el documento no ha sido modificado desde que se leyó, pero sería 1 si el documento se modificó):

In [ ]:
result = posts.replace_one({"_id": post['_id']}, post)
result.modified_count

## Búsqueda de documentos

Vamos a estudiar cómo buscar documentos en una colección de la BD.

En primer lugar con `.find_one` podemos buscar el primer elemento que cumpla una condición.


In [ ]:
post = posts.find_one()
pp(post)

In [ ]:
for k,v in post.items():
    print("%s: %s" % (k,v))

Además de `find_one()`, la función principal de búsqueda es `find()`. Esta función ofrece un conjunto muy ámplio de opciones para búsqueda, que estudiaremos a continuación.

Primero, una consulta sencilla, con el valor de un campo:

In [ ]:
posts.count_documents({'PostTypeId':2})

También existe `explain()`, al estilo de SQL.

In [ ]:
posts.find({"PostTypeId": 2}).explain()

También se puede limitar la búsqueda con la función `.limit`

In [ ]:
respuestas = posts.find({'PostTypeId': 2}).limit(10)

La respuesta no es un conjunto de elementos, sino un cursor que puede ir recorriéndose.

In [ ]:
respuestas

In [ ]:
list(respuestas)

También se puede importar en un Dataframe de `pandas`:

In [ ]:
respuestas = posts.find({"PostTypeId": 2}).limit(30)
df = pd.DataFrame(respuestas)
df.head()

In [ ]:
df['Id'].plot()

### Búsquedas anidadas

La función `find()` tiene un gran número de posibilidades para especificar la búsqueda. Se pueden utilizar cualificadores complejos como:

- `$and`
- `$or`
- `$not`

Estos calificadores unen "objetos", no valores. Por otro lado, hay otros calificadores que se refieren a valores:

- `$lt` (menor)
- `$lte` (menor o igual)
- `$gt` (mayor)
- `$gte` (mayor o igual)

In [ ]:
respuestas = posts.find({ '$and' : [ {"PostTypeId": 2} ,
                                    {"Id" : {'$gte' : 100}} ]}).limit(10)
list(respuestas)

Vamos ahora a obtener aquellas cuestiones que tengan un `Score`mayor a 100

In [ ]:
respuestas = posts.find({ '$and' : [ {"PostTypeId": 1} ,
                                    {"Score" : {'$gt' : 100}} ]}).limit(10)
list(respuestas)

Con el método `.sort` podemos ordenar los resultados en base a un campo. Dependiendo del valor del segundo parámetros tendremos un orden
* Ascendente (1)
* Descendente (-1)

In [ ]:
respuestas = posts.find({ '$and' : [ {"PostTypeId": 1} ,
                                    {"Score" : {'$gt' : 100}} ]}) \
                  .limit(10) \
                  .sort("LastActivityDate",1)
list(respuestas)

### Búsqueda por fechas

Es posible realizar búsquedas por fecha cuando los documentos incluyan atributos de ese tipo.

Para ello podemos usar `datetime` a la hora de definir los filtros.

In [ ]:
start = datetime(2021, 9, 29)
end = datetime(2023, 9, 24)

respuestas = posts.find({
    '$and' : [
        {"LastActivityDate": {'$gte':start,'$lt': end}}
        ]
    }).limit(10)
list(respuestas)

### Búsqueda con una lista

Con el operador `$in` podemos comprobar si el valor de un campo se encuentra dentro de una lista.

Vamos a obtener todos los documentos que tengan entre sus `Tags` los lenguajes *javascript* o *java*.

In [ ]:
respuestas = posts.find({'Tags': {'$in':['<javascript>', '<java>']}}).limit(3)
list(respuestas)

### Búsqueda con expresiones regulares

Aparte de los operadores visto arriba también es posible usar expresiones regulares con el operador `$regex`

*Más info sobre expresiones regulares en la [documentación oficial de Python](https://docs.python.org/3/howto/regex.html).*

Vamos extraer todas las preguntas de los usuarios cuyo título empiece por *python* o *Python*.

In [ ]:
respuestas = posts.find({ '$and' : [ {"PostTypeId": 1} ,
                                    {"Title" : {'$regex' : '^[Pp]ython'}} ]})
list(respuestas)

## Borrado de documentos

A la hora de realizar el borrado de documentos debemos de usar los métodos `.delete_one`o `.delete_many` usando filtros de forma similar a como hemos empleado en las búsquedas.

Vamos a eliminar aquellas respuestas que tengan un `Score` menor a 10.

In [ ]:
"""
resultado = posts.delete_many({ '$and' : [ {"PostTypeId": 2} ,
                                    {"Score" : {'$lt' : 10}} ]})
print(f'Borrados {resultado.deleted_count} documentos')
"""

## Inserción de elementos

A la hora de insertar elementos en MongoDB pude hacerse con los métodos `.insert_one` (para insertar un único documento) o `insert_many`para insertar múltiples documentos al mismo tiempo.



In [ ]:
post_to_insert = posts.find_one()
post_to_insert['Body']='Texto de prueba'
pp(post_to_insert)

In [ ]:
posts.insert_one(post_to_insert)

¿Por qué ha fallado? Si comprobamos detenidamente el documento, vemos que contiene el campo `_id` por lo que MongoDB nos informa que ya hay un documento con ese identificador. Por tanto, debemos de eliminar dicho campo del documento antes de insertarlo como uno nuevo.

In [ ]:
post_to_insert.pop('_id',None) #Borramos la clave '_id'.
posts.insert_one(post_to_insert)

## Operadores de colección

También hay operaciones específicas de la colección, como
* `.count_documents` y
* `.distinct`:

![distinct.bakedsvg.svg](https://github.com/dsevilla/bdge-data/raw/master/misc/mongo-distinct.png)

Podemos contar el número de elementos en una colección que cumplan una determinada condición

In [ ]:
db.posts.count_documents({'ViewCount':{'$gt':10000}})

Vamos a obtener los diferentes valors de `score` de los posts en la colección.

In [ ]:
db.posts.distinct('Score')

La función `$group` se usa dentro del *pipeline* de agregación de documentos seguido por Mongo (`.aggregate`). Esta función admite dos parámetros diferentes:
* `_id`: El identificador por el que queremos agrupar los documentos.
* `campo`: Expresión mediante la cual queremos aggregar los documentos (*opcional*).



Vamos a contar el número de posts por `owneruserid` en la colección `posts`.

In [ ]:
users_avg_scores= db.posts.aggregate(
    [{
        "$group":{
            "_id": "$OwnerUserId",
            "count": {"$sum":1}
        }
    }]
)

for u in users_avg_scores:
  print(u)

Ahora vamos computar el `score` medio por usuario en base a todos sus posts.

In [ ]:
users_avg_scores= db.posts.aggregate(
    [{
        "$group":{
            "_id": "$OwnerUserId",
            "avg_score": {"$avg":'$Score'}
        }
    }]
)

for u in users_avg_scores:
  print(u)

## Validación con `JSON` schema

En determinadas ocasiones puede ser interesante el *forzar* que los documentos de una colección cumplan un determinado `JSON schema`.

Vamos a crear una nueva colección `libro` y vamos a asociarle un `jsonschema` particular. Es importante destacar que dicho esquema fuerza a que el campo `autores` sea un array de otros objetos.

Para insertar el esquema en la colección debemos de ejecutar el comando `collMod` de MongoDB.

In [ ]:
def create_coleccion_libros():

    # Creamos una nueva colección
    try:
        db.create_collection("libro")
    except Exception as e:
        print(e)

    libro_validador = {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["titulo", "autores", "fecha_publicacion", "tipo", "copias"],
            "properties": {
                "titulo": {
                    "bsonType": "string",
                    "description": "debe de ser un string y es obligatorio"
                },
                "autores": {
                    "bsonType": "array",
                    "description": "debe de ser un array y es obligatorio",
                    "items": {
                        "bsonType": "objectId",
                        "description": "debe ser un objectId y es obligatorio"
                    },
                    "minItems": 1,
                },
                "fecha_publicacion": {
                    "bsonType": "date",
                    "description": "must be a date and is required"
                },
                "tipo": {
                    "enum": ["tapa_dura", "tapa_blanda"],
                    "description": "solo puede tomar los valores del enum y es obligatorio"
                },
                "copias": {
                    "bsonType": "int",
                    "description": "debe de ser un integer mayor que 0 y es obligatorio",
                    "minimum": 0
                }
            }
        }
    }

    db.command("collMod", "libro", validator=libro_validador)

create_coleccion_libros()

Ahora vamos a crear la colección de con los autores

In [ ]:
def crear_coleccion_autor():
    try:
        db.create_collection("autor")
    except Exception as e:
        print(e)

    autor_validator = {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["nombre_propio", "apellido"],
            "properties": {
                "nombre_propio": {
                    "bsonType": "string",
                    "description": "debe de ser un string y es obligatorio"
                },
                "apellido": {
                    "bsonType": "string",
                    "description": "debe de ser un string y es obligatorio"
                },
                "fecha_de_nacimiento": {
                    "bsonType": "date",
                    "description": "debe ser un date"
                }
            }
        }
    }

    db.command("collMod", "autor", validator=autor_validator)

crear_coleccion_autor()

Ahora podemos verificar las validaciones de ambas colecciones

In [ ]:
print(f'Validación de book: {db.get_collection("libro").options()}')
print(f'Validación de autor: {db.get_collection("autor").options()}')

Vamos a insertar una serie de documentos en ambas colecciones que cumplan con el esquema dado.

In [ ]:
def insertar_datos_validados():
    autores = [
        {
            "nombre_propio": "John",
            "apellido": "Doe",
            "fecha_de_nacimiento": datetime(1990, 1, 20)
        },
        {
            "nombre_propio": "Jane",
            "apellido": "Doe",
            "fecha_de_nacimiento": datetime(1990, 1, 1)
        },
        {
            "nombre_propio": "Jack",
            "apellido": "Smith",
        }
    ]

    autor_coll = db.autor
    autor_ids = autor_coll.insert_many(autores).inserted_ids
    print(f"IDs de los autores insertados: {autor_ids}")

    libros = [
        {
            "titulo": "MongoDB, The Book for Beginners",
            "autores": [autor_ids[0], autor_ids[1]],
            "fecha_publicacion": datetime(2022, 12, 17),
            "tipo": "tapa_dura",
            "copias": 10
        },
        {
            "titulo": "MongoDB, The Book for Advanced Users",
            "autores": [autor_ids[0], autor_ids[2]],
            "fecha_publicacion": datetime(2023, 1, 2),
            "tipo": "tapa_blanda",
            "copias": 5
        },
        {
            "titulo": "MongoDB, The Book for Experts",
            "autores": [autor_ids[1], autor_ids[2]],
            "fecha_publicacion": datetime(2023, 1, 2),
            "tipo": "tapa_blanda",
            "copias": 5
        },
        {
            "titulo": "100 Projects in Python",
            "autores": [autor_ids[0]],
            "fecha_publicacion": datetime(2022, 1, 2),
            "tipo": "tapa_dura",
            "copias": 20
        },
        {
            "titulo": "100 Projects in JavaScript",
            "autores": [autor_ids[1]],
            "fecha_publicacion": datetime(2022, 1, 2),
            "tipo": "tapa_blanda",
            "copias": 15
        }
    ]

    libro_coll = db.libro
    libros_insertados= libro_coll.insert_many(libros)

    print(f"Resultados de los libros insertados: {libros_insertados}")
    print(f"IDs de los libros insertados: {libros_insertados.inserted_ids}")

insertar_datos_validados()

Por último, si ahora se intentara insertar un libro que NO cumple con el esquema dado puesto que no incluye todos los campos obligatorios que tiene que tener un documento en dicha colección.

In [ ]:
db.libro.insert_one({
    "title": "MongoDB, The Book"
})

Vemos que la operación arroja una excepción `WriteError` al no cumplir con el esquema de la colección.

## Ejercicios

### EJERCICIO 1: Separar en dos colecciones las preguntas de las respuestas

### EJERCICIO 2: Crear una nueva colección `posts_premium` que incluye sólo aquellas preguntas creadas en 2022. Igualmente se deberá incluir la respuesta aceptada (`AcceptedAnswerId`) siempre y cuando su `Score` asignado sea mayor o igual a 1

¡Eso es todo amigos!